In [ ]:
%cd ..

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
algo_models = [
    # ("dmsp", "dae_0.04314"),
    ("dmsp", "dcnn_0.05"),
    ("dmsp", "drcnn_0.05"),
    # ("dmsp", "dunet_0.05"),
    ("dmsp", "dunet+_0.05"),
    # ("dmsp", "drunet+_0.05"),
    ("dmsp", "drunet+_0.0-0.2"),
    ("dmsp", "drugan+_0.0-0.2"),
    # ("hqs", "drunet_0.0-0.2"),
    ("hqs", "drunet+_0.0-0.2"),
    ("hqs", "drugan+_0.0-0.2"),
]
models = list(list(zip(*algo_models))[1])
metrics = ["psnr", "fsim"]
sr_scales = [2, 3, 4, 5]
datasets = ["set5-test"]

In [ ]:
# Read the evaluation
df_all = pd.read_csv('results/eval_image_reconstruction/super_res.csv')

# Read the model name mapping
with open('models/model_name_mapping.json') as f:
    model_mapping = json.load(f)

In [ ]:
df = df_all.copy()

# Only the final result
df = df[df['Step'] == df['num_steps'] - 1].copy()

# Only take rows where the model+algorithm combination is correct
df = df[[(a in algo_models) for a in zip(df['algorithm'], df['model'])]]
# df = df[df['model'].map(models_algorithm_dict) == df['algorithm']]

# Combine dataset and kernels name
df.loc[:, 'dataset'] = df['dataset'] + '-' + df['split']
df = df.drop(columns=['split', 'Step', 'Unnamed: 0', 'num_steps', 'sr_antialias'])

# Filter datasets
df = df[df['dataset'].isin(datasets)]

# Mean over the images
df = df.groupby(['model', 'sr_scale', 'algorithm']).mean()
df = df.reset_index()
df = df.drop(columns=['Image_Id'])

# Combine all metrics in one column
df = df.melt(id_vars=['model', 'sr_scale', 'algorithm'], var_name='metric', value_name='value')

# Filter: Only configured metrics, only configured noise levels
df = df[df['metric'].isin(metrics)]
df = df[df['sr_scale'].isin(sr_scales)]

# Sort
df = df.sort_values('metric', kind='mergesort', key=lambda x: [0 if v == "PSNR" else 1 for v in x])
df = df.sort_values('sr_scale', kind='mergesort')

# Rename metrics
df['metric'] = df['metric'].map(lambda x: x.upper())
df['algorithm'] = df['algorithm'].map(lambda x: x.upper())

# Set the index
df = df.set_index(['model', 'algorithm', 'sr_scale', 'metric'])

# Unstack sr_scale
df = df.unstack(['sr_scale', 'metric'])

# Organize the column naming
df.columns = df.columns.droplevel(0)
df.columns = df.columns.rename(['Scaling factor \( s \)', ''])

# Sort the models+algos
def index_level(n):
    return df.index.get_level_values(n)

df = df.sort_index(key=lambda x: [
    algo_models.index(x) for x in zip(index_level('algorithm').str.lower(),
                                      index_level('model'))
])

# Organize the index naming
df.index = df.index.map(lambda x: f"{x[1]}-{model_mapping[x[0]]}")
df.index = df.index.rename('Method')

df

In [ ]:
def float_format(v):
    if v < 1.0:
        return f"{v:.4f}" #[1:] <-- Do not show leading 0
    else:
        return f"{v:.2f}"

column_format = 'l' + 'c' * len(df.columns)

latex = df.to_latex(float_format=float_format,
                    escape=False,
                    na_rep='-',
                    column_format=column_format,
                    multicolumn_format='c')
print(latex)